In [387]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [388]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [389]:
S = (25, 10)
K = 2
OUT_DIM = 256
HID_DIM = 512
EPOCHS = 10

Download Dataset and create mini-batching of training data

In [390]:
from torch_geometric.datasets import Planetoid
#Download Pubmed dataset and save in data
dataset = Planetoid(root="data", name="Pubmed")
data = dataset[0]
setattr(data, "node_idx", torch.arange(data.num_nodes))

adj_list = [[] for _ in range(data.num_nodes)]
for edge in data.edge_index.T:
        list.append(adj_list[edge[0].item()], edge[1].item())
        list.append(adj_list[edge[1].item()], edge[0].item())
adj_list = adj_list
data.x

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.1046, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0194, 0.0080,  ..., 0.0000, 0.0000, 0.0000],
        [0.1078, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0266, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [391]:
#Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Training nodes: {sum(data.train_mask).item()}')
#print(f'Training nodes: {next(iter(train_batches))[1].shape}')
print(f'Evaluation nodes: {sum(data.val_mask).item()}')
#print(f'Evaluation nodes: {len(label["val"])}')
print(f'Testing nodes: {sum(data.test_mask).item()}')
#print(f'Testing nodes: {len(label["test"])}')


Dataset: Pubmed()
-------------------
Number of graphs: 1
Number of nodes: 19717
Number of features: 500
Number of classes: 3

Graph:
------
Training nodes: 60
Evaluation nodes: 500
Testing nodes: 1000


In [392]:
#mini-batching of training data
train_batches = DataLoader(TensorDataset(data.node_idx[data.train_mask], data.x[data.train_mask], data.y[data.train_mask]), batch_size=16)

Neighborhood Sampler

In [393]:
import random
""" 
Neighborhood Sampler
Params:
    node index of node the neighborhood should be sampled of
Returns:
    index list of nodes in neigborhood of node
"""
def sample(batch):
    neigh_hop1 = []
    neigh_hop2 = []
    for node in batch:
        neigh_hop1.append(torch.tensor(random.choices(adj_list[node], k=S[0])))
    hop1_idx = torch.unique(torch.cat([batch, torch.cat(neigh_hop1)]))
    for node in hop1_idx:
        neigh_hop2.append(torch.tensor(random.choices(adj_list[node], k=S[1])))
    hop2_idx = torch.unique(torch.cat([hop1_idx, torch.cat(neigh_hop2)]))
    return (hop2_idx, hop1_idx, batch), (neigh_hop1 + neigh_hop2, neigh_hop1)

B, N = sample(data.node_idx[data.train_mask])
print(B[0].shape)

torch.Size([1290])


Aggregator function

In [394]:
class MaxPoolingAggregator(torch.nn.Module):
    """ 
    AGGREGATOR: Max Pooling
    Params:
        in_channels feature size of each input sample
        out_channel feature size of each output sample
    """
    def __init__(self, in_channels, out_channels):
        super(MaxPoolingAggregator, self).__init__()
        #fully connected layer with learnable weights
        self.fc_layer = torch.nn.Linear(in_channels, out_channels, bias=True)
    """
    Forward Propagation
    Params:
        neighborhood neigborhood sample of imput node to be aggregated
    feed neighborhood of node through fully connected layer and non linearity
    return maximum of all neighbors
    """
    def forward(self, neighbors):
        out = []
        for h in neighbors:
            h = self.fc_layer(h).relu()
            out.append(h)
        return torch.amax(torch.stack(out), 0).requires_grad_()

Graph SAGE model

In [395]:
class GraphSAGE(torch.nn.Module):
    """ 
    GraphSAGE model
    Params:
        in_channels feature size of each input sample
        hidden_channels feature size of each hidden sample
        out_channel feature size of each output sample
        num_layers =K number of message passing layers
    """
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super().__init__()
        self.aggregate = torch.nn.ModuleList([MaxPoolingAggregator(in_channels, hidden_channels), 
                                              MaxPoolingAggregator(in_channels, hidden_channels)])
        self.linears = torch.nn.ModuleList([torch.nn.Linear(in_channels + hidden_channels, out_channels, bias=False), 
                                            torch.nn.Linear(out_channels + hidden_channels, out_channels, bias=False)])
        #Use Adam optimizer vgl Experimental Setup
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.1)
        #Cross entropy loss for supervised learning 
        self.loss_fn = F.cross_entropy
        self.lin_out = torch.nn.Linear(out_channels, num_classes, bias=False)

    """
    Forward Propagation
    Params:
        x data object (node indeces, node features, node lable)

    """
    def forward(self, x, idx):
        #neighbors and lists of neighbors in each iteration
        B, N = sample(idx)
        #vectors of embeddings ????
        z = [data.x[B[0]]]
        for k in range(K):
            z.append(torch.empty(len(B[k+1]), OUT_DIM))
            for i, u in enumerate(B[k+1]):
                h_n = self.aggregate[k](data.x[N[k][i]])
                h = torch.cat((z[k][i], h_n))
                h = self.linears[k](h).relu()
                h = F.normalize(h, dim=-1)
                z[k+1][i] = h
        return self.lin_out(z[K])


In [399]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    model.eval()
    out = model(data.x[data.test_mask], data.node_idx[data.test_mask])
    acc = accuracy(out.argmax(dim=1), data.y[data.test_mask])
    return acc

In [397]:
def train(model):
    model.train()
    for epoch in range(EPOCHS+1):
        total_loss = 0
        acc = 0

        for batch in train_batches:
            model.optimizer.zero_grad()
            out = model(batch[1], batch[0])
            loss = model.loss_fn(out, batch[2])
            total_loss += loss
            acc += accuracy(out.argmax(dim=1), batch[2])
            loss.backward()
            model.optimizer.step()

        print(f'Epoch {epoch:>3} | Train Loss: {loss/len(train_batches):.3f} '
          f'| Train Acc: {acc/len(train_batches)*100:>6.2f}%')

In [400]:
model = GraphSAGE(data.num_node_features, HID_DIM, OUT_DIM, dataset.num_classes)
print(model)

train(model)

print(f'\nGraphSAGE test accuracy: {test(model, data)*100:.2f}%\n')

GraphSAGE(
  (aggregate): ModuleList(
    (0): MaxPoolingAggregator(
      (fc_layer): Linear(in_features=500, out_features=512, bias=True)
    )
    (1): MaxPoolingAggregator(
      (fc_layer): Linear(in_features=500, out_features=512, bias=True)
    )
  )
  (linears): ModuleList(
    (0): Linear(in_features=1012, out_features=256, bias=False)
    (1): Linear(in_features=768, out_features=256, bias=False)
  )
  (lin_out): Linear(in_features=256, out_features=3, bias=False)
)
Epoch   0 | Train Loss: 0.342 | Train Acc:  26.56%
Epoch   1 | Train Loss: 0.335 | Train Acc:  31.25%
Epoch   2 | Train Loss: 0.328 | Train Acc:  37.50%
Epoch   3 | Train Loss: 0.317 | Train Acc:  50.00%
Epoch   4 | Train Loss: 0.296 | Train Acc:  62.50%
Epoch   5 | Train Loss: 0.265 | Train Acc:  62.50%
Epoch   6 | Train Loss: 0.252 | Train Acc:  87.50%
Epoch   7 | Train Loss: 0.243 | Train Acc:  97.92%
Epoch   8 | Train Loss: 0.238 | Train Acc:  97.92%
Epoch   9 | Train Loss: 0.235 | Train Acc: 100.00%
Epoch  10